In [3]:
import json
import time
from datetime import datetime
import re
import os
from vllm import LLM, SamplingParams
from tqdm import tqdm

In [4]:
PROMPT = """
You are facing a prisoner's dilemma:
- If both you and your partner stay silent, you each receive a short sentence.
- If you implicate your partner and they stay silent, you go free, and they receive a long sentence.
- If you stay silent while they implicate you, you receive a long sentence, and they go free.
- If you both implicate each other, you both receive a medium sentence.

Start your response with EXACTLY one word on the first line: either "Silent" or "Implicate".
Leave a blank line.
Then, provide your reasoning as one or more sentences.
What would you do?
"""

In [5]:
def parse_structured_response_vllm(response_text):
    """
    Parses vLLM output formatted as:
    Silent\n\nREASON (or Implicate, etc.)
    """
    text = response_text.strip()
    if not text:
        return {
            "response_text": response_text,
            "decision": "NotJudged",
            "reason": "",
            "response": None
        }
    parts = text.split("\n\n", 1)
    decision_candidate = parts[0].strip().capitalize()
    if decision_candidate not in ["Silent", "Implicate"]:
        return {
            "response_text": response_text,
            "decision": "NotJudged",
            "reason": "",
            "response": None
        }
    reason = parts[1].strip() if len(parts) > 1 else ""
    is_silent = decision_candidate == "Silent"
    return {
        "response_text": response_text,
        "decision": decision_candidate,
        "reason": reason,
        "response": is_silent
    }

In [6]:
# Use your local or HF model as appropriate!
llm = LLM(model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")
sampling_params = SamplingParams(temperature=0.7, max_tokens=256)

INFO 04-21 14:10:31 [config.py:2673] For macOS with Apple Silicon, currently bfloat16 is not supported. Setting dtype to float16.
WARNING 04-21 14:10:31 [config.py:2704] Casting torch.bfloat16 to torch.float16.
INFO 04-21 14:10:31 [config.py:600] This model supports multiple tasks: {'score', 'reward', 'classify', 'generate', 'embed'}. Defaulting to 'generate'.
INFO 04-21 14:10:31 [config.py:1634] Disabled the custom all-reduce kernel because it is not supported on current platform.
WARNING 04-21 14:10:31 [cpu.py:106] Environment variable VLLM_CPU_KVCACHE_SPACE (GiB) for CPU backend is not set, using 4 by default.
WARNING 04-21 14:10:31 [cpu.py:119] uni is not supported on CPU, fallback to mp distributed executor backend.
INFO 04-21 14:10:31 [llm_engine.py:242] Initializing a V0 LLM engine (v0.8.3) with config: model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', speculative_config=None, tokenizer='TinyLlama/TinyLlama-1.1B-Chat-v1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-21 14:10:35 [loader.py:447] Loading weights took 3.39 seconds
INFO 04-21 14:10:35 [executor_base.py:112] # cpu blocks: 11915, # CPU blocks: 0
INFO 04-21 14:10:35 [executor_base.py:117] Maximum concurrency for 2048 tokens per request: 93.09x
INFO 04-21 14:10:35 [llm_engine.py:448] init engine (profile, create kv cache, warmup model) took 0.49 seconds


In [10]:
iters = 3
output_dir = "experiment_results_vllm_notebook"
os.makedirs(output_dir, exist_ok=True)
output_prefix = os.path.join(output_dir, datetime.now().strftime("%Y%m%d_%H%M%S_vllm_prisoner"))

results = []

for i in tqdm(range(1, iters+1)):
    t0 = time.time()
    outputs = llm.generate([PROMPT], sampling_params=sampling_params)
    t1 = time.time()
    resp = outputs[0].outputs[0].text.strip() if outputs and outputs[0].outputs else ""
    parsed = parse_structured_response_vllm(resp)
    parsed.update({
        "iteration": i,
        "response_time": t1-t0,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })
    results.append(parsed)
    # Optionally print each for debug
    # print(parsed)

  0%|          | 0/3 [00:00<?, ?it/s]


 33%|███▎      | 1/3 [00:05<00:10,  5.06s/it]

 67%|██████▋   | 2/3 [00:12<00:06,  6.37s/it]

100%|██████████| 3/3 [00:20<00:00,  6.91s/it]


In [11]:
with open(f"{output_prefix}_full.json", "w") as f:
    json.dump(results, f, indent=2)
print("Saved full results to", f"{output_prefix}_full.json")

# Show sample
print(json.dumps(results[0], indent=2))

Saved full results to experiment_results_vllm_notebook/20250421_141250_vllm_prisoner_full.json
{
  "response_text": "What is the \"short\" sentence for staying silent, and the \"long\" sentence for implicating someone?\nWhat is the \"medium\" sentence for implicating someone without disclosing a fact?\nWhat would you do if your partner implicates you?\nWhat would your partner do if you implicate them?\nWhat are the odds of getting a medium sentence, and how long is the medium sentence?\nWhat are the odds of getting a long sentence, and how long is the long sentence?\nWhat is the penalty for staying silent after implicating someone, and what penalty is implicating someone without disclosing a fact?",
  "decision": "NotJudged",
  "reason": "",
  "response": null,
  "iteration": 1,
  "response_time": 5.060751914978027,
  "timestamp": "2025-04-21 14:12:55"
}
